<a href="https://colab.research.google.com/github/cartmarsh/MachineLearning1/blob/main/Toxic_comment_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Outline

1. Get the tweets with tweepy
2. load the model
3. feed it the tweets

4. Make some Data Analysis

In [1]:
!pip install tweepy==4.1.0
!pip install python-dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 740 kB/s 
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install textblob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install wordcloud
!pip install better_profanity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 46 kB 2.7 MB/s 


In [4]:
import re 

import numpy as np

import tweepy 

from tweepy import OAuthHandler 

from textblob import TextBlob 

import matplotlib.pyplot as plt

import pandas as pd

from wordcloud import WordCloud

from better_profanity import profanity

In [9]:
consumer_key = 'EaPXlI8mJH7ZRGWwajcI0bMtu'
consumer_secret = 'MnLvrs6HxDRHB2MGYlnlrlbufl1eoZgqieYGE9Hbg1ipw4dV0I' 
access_token = '840693273345822720-wmQuqwFOvu6iWIHn33D1podAIzL1ZeV'
access_token_secret = 'hy7u64l95h5Fog2GoRpQzFURwgS0hMWdWfnGCjFlmMyOK'

In [10]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)


api = tweepy.API(auth)

In [11]:
tweepy.__version__

'4.1.0'

In [12]:
# Input a query from the user

query = input("Please enter your topic / person of interest: ")

# In this case, we will input the query as 'Elon Musk'

# Filter the query to remove retweets

filtered = query + "-filter:retweets"

# Generate the latest tweets on the given query 

tweets = tweepy.Cursor(api.search_tweets, 
                           q=filtered,
                           lang="en").items(100)

# Create a list of the tweets, the users, and their location

list1 = [[tweet.text, tweet.user.screen_name, tweet.user.location] for tweet in tweets]

Please enter your topic / person of interest: musk


In [19]:
# Convert the list into a dataframe

df = pd.DataFrame(data=list1, 
                    columns=['tweets','user', "location"])
# Convert only the tweets into a list

tweet_list = df.tweets.to_list()

In [20]:
tweet_list[:10]

['@eyishazyer @wise_chimp And dedication will ensure that you maintain your consistency.',
 '@PueblaAyto @Investingcom Musk forever!',
 '@KobaltJew I mean musk is just out there switching things on and off trying to see what breaks it so…🤷🏻\u200d♀️',
 '@PueblaAyto @litecoin Musk forever!',
 '@PueblaAyto @litecoin Musk, this is best event on 2022!',
 'My account was permanently suspended for over a month and I thought I’ll never get it back. But thanks for the 26th… https://t.co/79Bo0Zzqy8',
 "What an EXCEPTIONAL 2022 for America's national security state!\n\nRFK Jr. CONFIRMS CIA killed JFK.\n\nElon Musk CONFIR… https://t.co/pEJc8WQqA4",
 "@benshapiro But that's what Musk did, he never did anything himself, he just bought talent!",
 'Elon Musk Sells Another Big Chunk of Tesla Stock\n\n#Tesla #Twitter https://t.co/nV1RcFCQnY',
 '@UberFacts Because Elon Musk is the greatest thing that happened to humanity']

We can improve the stopwords here by donwloading a library

In [21]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer

In [22]:
stemmer = SnowballStemmer(language='english')

In [23]:
nltk.download('stopwords')
from nltk.corpus import stopwords
english_stopwords = stopwords.words('english')
", ".join(english_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [24]:
def clean_tweet(tweet):
    if type(tweet) == float:
        return ""
    r = tweet.lower()
    r = profanity.censor(r)
    r = re.sub("'", "", r) # This is to avoid removing contractions in english
    r = re.sub("@[A-Za-z0-9_]+","", r)
    r = re.sub("#[A-Za-z0-9_]+","", r)
    r = re.sub(r'http\S+', '', r)
    r = re.sub('[()!?]', ' ', r)
    r = re.sub('\[.*?\]',' ', r)
    r = re.sub("[^a-z0-9]"," ", r)
    r = r.split()
    stopwords = english_stopwords
    r = [w for w in r if not w in stopwords]
    r = " ".join(word for word in r)
    return r

In [25]:
cleaned = [clean_tweet(tw) for tw in tweet_list]
cleaned[:10]

['dedication ensure maintain consistency',
 'musk forever',
 'mean musk switching things trying see breaks',
 'musk forever',
 'musk best event 2022',
 'account permanently suspended month thought never get back thanks 26th',
 'exceptional 2022 americas national security state rfk jr confirms cia killed jfk elon musk confir',
 'thats musk never anything bought talent',
 'elon musk sells another big chunk tesla stock',
 'elon musk greatest thing happened humanity']

Load the model

In [26]:
import torch

In [27]:
!pwd


/content


In [28]:
!ls

kaggle.json  sample_data


In [30]:
# model = torch.load("./checkpointRNN_toxic_comment_classifier.ckpt")

In [33]:
!pip install pytorch_lightning --quiet

In [32]:
import pytorch_lightning as pl

Prepare the data for prediction

In [34]:
from torchtext.data.utils import get_tokenizer

In [35]:
tokenizer = get_tokenizer("basic_english")

In [36]:
from torchtext.vocab import build_vocab_from_iterator

In [37]:
comments_token = [tokenizer(comment) for comment in cleaned]

In [38]:
comments_token = pd.DataFrame(comments_token)

In [39]:
comments_token[:10]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,dedication,ensure,maintain,consistency,None,None,None,None,None,None,None,None,None,None,None
1,musk,forever,None,None,None,None,None,None,None,None,None,None,None,None,None
2,mean,musk,switching,things,trying,see,breaks,None,None,None,None,None,None,None,None
3,musk,forever,None,None,None,None,None,None,None,None,None,None,None,None,None
4,musk,best,event,2022,None,None,None,None,None,None,None,None,None,None,None
5,account,permanently,suspended,month,thought,never,get,back,thanks,26th,None,None,None,None,None
6,exceptional,2022,americas,national,security,state,rfk,jr,confirms,cia,killed,jfk,elon,musk,confir
7,thats,musk,never,anything,bought,talent,None,None,None,None,None,None,None,None,None
8,elon,musk,sells,another,big,chunk,tesla,stock,None,None,None,None,None,None,None
9,elon,musk,greatest,thing,happened,humanity,None,None,None,None,None,None,None,None,None


In [40]:
VOCAB_SIZE = 1500

In [41]:
## for words that are unknown to the vocab
unk_token = ""

## to pad sentences, because we need the same length for RNNs
pad_token = ""

In [42]:
vocab = build_vocab_from_iterator(comments_token, specials=[unk_token, pad_token],
                                  max_tokens=VOCAB_SIZE)

TypeError: ignored

In [43]:
from torch.utils.data import DataLoader